In [ ]:

import os
import pandas as pd
import openai
import time
from sklearn.metrics import classification_report

openai.api_key = ''

folder = r"Data\Multiple_Neg_Data"
global results2
print(folder)

C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Multiple_Neg_Data


In [50]:
def process_missing_keys(sentences,missingkeys,results,batch,batch_indices,batchnumber):
    print("i am here in process_missing_keys")
    m={}
    for key,indexlist in missingkeys.items():
        for index in indexlist:
            m[index]=sentences[index]
    
    batch_indices=list(m.keys())
    batch=list(m.values())
    print(batch_indices)
    print(batch)


    def process_batch(batch,batch_indices,batchnumber,results,missingkeys):
       
        print("results in start process_missing_keys function ", results)

        prompt = "\n".join([f"{i}. Classify the sentence '{batch[i]}' as Multiple Negation or Not Multiple Negation in one word which is a concept of AAVE English and keep the numbering given at the start of the prompt intact in the response."
                            for i in range(len(batch))])
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ]
            )

            responses_map = {}
            responses = [res.strip() for res in response.choices[0].message['content'].strip().split('\n')]
            print("Actual responses in process_missing_keys ", responses)

            lineindex=0
            for line in responses:
                if ". " in line:
                    try:
                        indexr, value = line.split(". ", 1)
                        indexr = int(indexr)
                        responses_map[batch_indices[indexr]] = value
                    except ValueError:
                        print(f"Malformed response line : '{line}'")
                        print("linindex:  ",lineindex)
                        if 'M' in line or 'm' in line or 'N' in line or 'n' in line:
                            responses_map[batch_indices[lineindex]]=line
                    lineindex+=1
                else:
                    print(f"Line does not contain expected delimiter: '{line}'")

            for i, idx in enumerate(batch_indices):
                if idx in responses_map:
                    results[idx] = responses_map[idx]
                    print("************************************",batchnumber,missingkeys,idx,results[idx])
                    del missingkeys[batchnumber]
                # else:
                #     if batchnumber not in missingkeys:
                #         missingkeys[batchnumber] = []
                #     missingkeys[batchnumber].append(idx)

            print("Responses Map in process_missing_keys ", results)
            print("Missing Keys in process_missing_keys", missingkeys)
            
           

        except openai.error.RateLimitError:
            print("Rate limit exceeded. Waiting to retry...")
            time.sleep(80)  # Wait for 80 seconds before retrying
        # except Exception as e:
        #     print(f"Error processing batch: {e}")
        return results,missingkeys
    return process_batch(batch,batch_indices,batchnumber,results,missingkeys)
        

In [51]:
# Function to generate prompts based on train data
# def generate_prompt(train_data):
#     examples = []
#     for _, row in train_data.iterrows():
#         sentence = row['Sentence']
#         label = "habitual be" if row['Habituality'] == 1 else "non-habitual be"
#         examples.append(f"Sentence: {sentence}\nLabel: {label}\n")
#     prompt = "\n".join(examples)
#     return prompt


In [52]:
def process_batch(sentences,batch, batch_indices, batchnumber):
    global results2
    missingkeys1={}
    prompt = "\n".join([f"{i}. Classify the sentence '{batch[i]}' as Multiple Negation or Not Multiple Negation in one word which is a concept of AAVE English and keep the numbering given at the start of the prompt intact in the response."
                            for i in range(len(batch))])
    # print("promt hereee  ",prompt)
    try:
        response = openai.ChatCompletion.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )

        responses_map = {}
        responses = [res.strip() for res in response.choices[0].message['content'].strip().split('\n')]
        print("Actual responses**********: ", responses)
        




        lineindex=0
        for line in responses:
            if ". " in line:
                try:
                    indexr, value = line.split(". ", 1)
                    indexr = int(indexr)
                    responses_map[batch_indices[indexr]] = value
                except ValueError:
                    print(f"Malformed response line : '{line}'")
                    print("linindex:  ",lineindex)
                    if 'M' in line or 'm' in line or 'N' in line or 'n' in line:
                        responses_map[batch_indices[lineindex]]=line
                lineindex+=1
            else:
                print(f"Line does not contain expected delimiter: '{line}'")
        # print("hererre testtt",responses_map)          
        for i, idx in enumerate(batch_indices):
            if idx in responses_map:
                results2[idx] = responses_map[idx]
            else:
                if batchnumber not in missingkeys1:
                    missingkeys1[batchnumber] = []
                missingkeys1[batchnumber].append(idx)

        # print("Responses Map:", responses_map)
        print("Missing Keys:", missingkeys1)
        print("resultss up",results2)

        if missingkeys1:
            print("did i come here")
            
            newresults,newmissingkeys=process_missing_keys(sentences,missingkeys1,results2,batch,batch_indices,batchnumber)
        # time.sleep(80)      
            missingkeys1=newmissingkeys
            results2=newresults
    except openai.error.RateLimitError:
        print("Rate limit exceeded. Waiting to retry...")
        time.sleep(80)  # Wait for 80 seconds before retrying
    # except Exception as e:
    #     print(f"Error processing batch: {e}")
        time.sleep(80)  # Wait for 80 seconds before retrying
    

    return results2,missingkeys1

In [53]:
# Function to classify sentences in batches
def classify_sentences_batch(sentences, batch_size=20):
    global results2
    results2 = {i: None for i in range(len(sentences))}  # Initialize results dictionary with None values
    # Process sentences in batches
    batchnumber = 1
    for i in range(0, len(sentences), batch_size):
        batch_indices = list(range(i, min(i + batch_size, len(sentences))))
        localresults,localmissingkeys=process_batch(sentences,[sentences[idx] for idx in batch_indices], batch_indices, batchnumber)
        # i=1
        # for key,value in localresults.items():
        #     if value==None:
        #         print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@ issuesss @@@@@@@@@  :::: ",key, value,batch_indices[key],batchnumber)
        #         i+=1
        # print(i)
        print("batch indicies",batch_indices)
        print("localreuslts  ",localresults)
        batchnumber += 1
    print("results downnnnnnn******************",results2)
    return results2


In [54]:
import re
def gatherResults(test_data,test_sentences,results):
    print("#################################################################  hcdjcnd")
    fold_predictions = []
    fold_true_labels = []
    for idx in range(len(test_sentences)):
            # Access prediction directly from the results list
            prediction = results[idx] if idx < len(results) else None
            prediction=re.sub(r'^[^a-zA-Z]+', '', prediction)

            if prediction:  # Check if prediction is not None
                prediction = prediction.lower()
                print("prediction******************************* ",prediction)
            if prediction is None:
                # Handle None values
                numeric_prediction = 0  # Assign a default numeric value (e.g., 0) for None predictions
            else:
                # Convert the prediction to numeric: 1 for "habitual be", -1 for "non-habitual be"
                if  prediction.startswith('m'):
                    numeric_prediction = 1
                elif prediction.startswith('n'):
                    numeric_prediction = 0
                else:
                    numeric_prediction = '*'  # Handle unexpected cases

            # Get the true label
            true_label = 1 if test_data.loc[idx, 'Mneg'] == 1 else 0

            fold_predictions.append(numeric_prediction)
            fold_true_labels.append(true_label)

            # print(f"Sentence {idx}: '{test_sentences[idx]}' -> Classification: {numeric_prediction} (True: {true_label})")
    print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^",fold_predictions)
    return fold_predictions,fold_true_labels
            


In [55]:
def interleave_labels(positive_data, negative_data):
    """
    Interleave sentences with label `1` and `-1` as much as possible and append remaining.
    """
    interleaved_data = []

    pos_len = len(positive_data)
    neg_len = len(negative_data)
    min_len = min(pos_len, neg_len)

    # Interleave `1`s and `-1`s one by one
    for i in range(min_len):
        interleaved_data.append(positive_data.iloc[i].to_dict())  # Convert row to dictionary
        interleaved_data.append(negative_data.iloc[i].to_dict())  # Convert row to dictionary

    # Append the rest of the remaining data
    if pos_len > neg_len:
        interleaved_data.extend(positive_data.iloc[min_len:].to_dict('records'))
    else:
        interleaved_data.extend(negative_data.iloc[min_len:].to_dict('records'))

    # Convert to DataFrame using the column names explicitly
    interleaved_df = pd.DataFrame(interleaved_data, columns=["Text", "Mneg"])
    return interleaved_df


In [56]:
#main code
from sklearn.utils import shuffle

completeresults=[]
data_file=r"C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Multiple_Neg_Data\test multiple negation.txt"
results_folder=r"C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Results_Data\M_Neg_Results"
def main():
    # Ensure the results folder exists
    # if not os.path.exists(folder):
    #     os.makedirs(folder)
    
    # Load the single text file
    print(f"Processing {data_file}")
    data = pd.read_csv(data_file, sep=',', header=None, names=["Text", "Mneg"])
    test_sentences = data['Text'].tolist()  # First column
    true_labels = data['Mneg'].tolist() 
   
   # Shuffle the data
    #--new code added shuffling----------------------------------------- 
#---------------------------------------------------------------------------------------------
    # data = shuffle(data, random_state=42)  # Shuffle data with a fixed random seed for reproducibility
    # test_sentences = data['Text'].tolist()  # First column
    # true_labels = data['Mneg'].tolist() 
#--------------------------------------------------------------------------
#all 1;s and -1s together-------------------------------------------------
    # positive_data = data[data['Mneg'] == 1]
    # negative_data = data[data['Mneg'] == 0]

    # # Concatenate to get all 1's followed by all -1's
    # grouped_data = pd.concat([positive_data, negative_data], ignore_index=True)

    # # Save the grouped data for verification
    # grouped_data_file = os.path.join(results_folder, 'grouped_test_multiple_negation.csv')
    # grouped_data.to_csv(grouped_data_file, index=False)
    # print(f"Grouped data saved to {grouped_data_file}")

    # test_sentences = grouped_data['Text'].tolist()
    # true_labels = grouped_data['Mneg'].tolist()
    # print(len(test_sentences))
#---------------------------------------------------------------------------


#------------------------------------------------------------------------------------------------
# Group all sentences with label 1 and -1 separately
    positive_data = data[data['Mneg'] == 1].reset_index(drop=True)
    negative_data = data[data['Mneg'] == 0].reset_index(drop=True)

    # Interleave sentences with label 1 and -1
    interleaved_data = interleave_labels(positive_data, negative_data)

    # Save the interleaved data for verification
    interleaved_data_file = os.path.join(results_folder, 'interleaved_test_multiple_negation.csv')
    interleaved_data.to_csv(interleaved_data_file, index=False)
    print(f"Interleaved data saved to {interleaved_data_file}")

    # Prepare the sentences and true labels after interleaving
    test_sentences = interleaved_data['Text'].tolist()
    true_labels = interleaved_data['Mneg'].tolist()

#------------------------------------------------------------------------------------------------

    
    # Print the first sentence and label for verification
    print(f"First shuffled sentence: {test_sentences[0]}")
    print(f"First shuffled label: {true_labels[0]}")
    
    
    # Print the first sentence and label for verification
    print(test_sentences[0])
    print(true_labels[0])
    # Classify sentences in batches
    results = classify_sentences_batch(test_sentences, batch_size=20)
    
    # Gather predictions and true labels
    fold_predictions, fold_true_labels = gatherResults(data, test_sentences, results)
    
    # Save results to a CSV file
    output_df = pd.DataFrame({
        'Sentence': test_sentences,
        'True_Label': fold_true_labels,
        'Prediction': fold_predictions
    })
    # output_file = os.path.join(results_folder, 'New_predictions_multiple_neg.csv')
    # output_file = os.path.join(results_folder, 'Random_shuffle_predictions_multiple_neg.csv')

    # output_file = os.path.join(results_folder, 'New_shuffled(0_and_1_together_predictions_multiple_neg.csv')
    output_file = os.path.join(results_folder, 'New_shuffled_interleaved_predictions_multiple_neg.csv')


    output_df.to_csv(output_file, index=False)
    print(f"Results saved to {output_file}")

    print("Complete prediction set:", len(fold_predictions), " True labels:", len(fold_true_labels))
    return fold_predictions, fold_true_labels
    return [],[]

In [57]:
#main run
all_predictions=[]
all_true_labels=[]
# all_predictions
all_predictions,all_true_labels=main()


Processing C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Multiple_Neg_Data\test multiple negation.txt
Interleaved data saved to C:\Users\gowda\OneDrive\Documents\UF Study\Jotsna docs\OPS\Updated Code\LLM\Claude AI\KFold Data\Data\Results_Data\M_Neg_Results\interleaved_test_multiple_negation.csv
First shuffled sentence: So we were didn't want for nothing.
First shuffled label: 1
So we were didn't want for nothing.
1
Actual responses**********:  ['0. Multiple Negation', '1. Not Multiple Negation', '2. Multiple Negation', '3. Not Multiple Negation', '4. Multiple Negation', '5. Not Multiple Negation', '6. Multiple Negation', '7. Not Multiple Negation', '8. Multiple Negation', '9. Not Multiple Negation', '10. Multiple Negation', '11. Not Multiple Negation', '12. Multiple Negation', '13. Not Multiple Negation', '14. Multiple Negation', '15. Not Multiple Negation', '16. Multiple Negation', '17. Not Multiple Negation', '18. Multiple Negat

In [58]:
print(len(all_predictions))
print(len(all_true_labels))

198
198


In [59]:
# print(all_predictions)
i=0
for index,pred in enumerate(all_predictions):
    if pred=='*':
        print(index, pred)
        i+=1
print(i)

0


In [60]:
print(classification_report(all_true_labels, all_predictions, target_names=['Non Multiple Neg (label 0)', 'Multiple Neg (label 1)']))


                            precision    recall  f1-score   support

Non Multiple Neg (label 0)       0.84      0.81      0.82       162
    Multiple Neg (label 1)       0.25      0.28      0.26        36

                  accuracy                           0.72       198
                 macro avg       0.54      0.55      0.54       198
              weighted avg       0.73      0.72      0.72       198



In [ ]:
completeresults